# Cell boundaries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
import numpy as np
import warnings
import geopandas as gpd
import tifffile
import os
import json
from fast_alphashape import alphashape
from shapely.ops import transform
import gc
warnings.filterwarnings('ignore')

## Import file

In [ ]:
def import_files() -> pd.DataFrame:
    df_vizgen = pd.read_csv("data/transcripts_cellpose.csv")
    df_vizgen = df_vizgen[['transcript_id', 'cell_id']]
    df_baysor = pd.read_csv("data/transcripts.csv")
    df = pd.merge(left=df_baysor, right=df_vizgen, how="left", left_on="transcript_id", right_on='transcript_id')
    df['cell_id'] = df.cell_id.replace(0, np.nan)
    return df

df = import_files()

In [ ]:
df

In [ ]:
def import_image(path: str, scale: int = 4):
    file = os.path.join(path, "image.tiff")
    with tifffile.TiffFile(file) as tif:
        page = tif.pages[0]
        height = (page.imagelength)
        width  =  (page.imagewidth)
        pad_height = (scale - height % scale) % scale
        pad_width = (scale - width % scale) % scale
        shape = (height + pad_height) // scale, (width + pad_width) // scale
        stack = np.empty(shape, page.dtype)
        stack = page.asarray(out='memmap')[::scale, ::scale]
        return stack

img = import_image("data/")
img

In [ ]:
def create_rescaling_function(scale=4.0):
    """
    Here we create a rescaling function that takes three argument: x, y, z.
    These are the coordinates of out point. We will apply the transformation
    defined in 'micron_to_mosaic_pixel_transform' to convert the micron to pixel
    """
    
    mmpt = pd.read_csv(
        "data/pixel_transformation.csv", 
        header=None, 
        sep=' '
    )
    mmpt = np.array(mmpt)

    def rescale_fun(x,y,z):
        points = np.array([x,y,z])
        # coordinates to pixel
        points = np.diag(mmpt) * points.T + np.append(mmpt[0:2,2], 0)
        points = points / scale
        return(points.T)
    
    return(rescale_fun)

rescale_fun = create_rescaling_function()
def rescale_fun_2d(x,y):
    ones = np.ones(len(x))
    res = rescale_fun(x,y,ones)

    return(res[0],res[1])

## Subset to a smaller FOV

Larger slides might take a long time to plot and the image would be too crowded to actually see the boundaries. Hence, we subset it to a smaller field of view (FOV)

In [ ]:
max_width = int(os.getenv("WIDTH"))
max_height = int(os.getenv("HEIGHT"))

x_offset = int(os.getenv("X_OFFSET"))
y_offset = int(os.getenv("Y_OFFSET"))

total_width = np.max(df.x) - np.min(df.x)
total_height = np.max(df.y) - np.min(df.y)

# check boundaries
if max_width >= total_width:
    max_width = total_width
    x_offset = 0
if max_height >= total_height:
    max_height = total_height
    y_offset = 0


if (x_offset < 0) and (total_width > max_width):
    x_offset = round(total_width /2 - max_width /2)
if (max_width + x_offset) > np.max(df.x):
    x_offset = np.max(df.x) - max_width


if (y_offset < 0) and (total_height > max_height):
    y_offset = round(total_height /2 - max_height /2)
if (max_height + y_offset) > np.max(df.y):
    y_offset = np.max(df.y) - max_height



In [ ]:
# transform FOX to pixel

def transform_rectangle(x, y, width, height):
    start = rescale_fun(x, y, 0)
    end = rescale_fun(x+width, y+height, 0)
    return (
        round(start[0]), 
        round(start[1]),
        round(end[0] - start[0]),
        round(end[1] - start[1])
    )

tx, ty, tw, th = transform_rectangle(x_offset, y_offset, max_width, max_height)
print(x_offset, y_offset, max_width, max_height)
print(tx, ty, tw, th)

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.imshow(
    img,
    vmin=np.percentile(img, 99)*0.1,
    vmax=np.percentile(img, 99)*1.1,
    cmap=sns.dark_palette("#bfcee3", reverse=False, as_cmap=True)
)
img_size = Rectangle((0,0),img.shape[1], img.shape[0], edgecolor='b', facecolor='none')
fov = Rectangle((tx, ty), tw, th, edgecolor='r', facecolor='none')
ax.add_patch(img_size)
ax.add_patch(fov)
ax.set_xlim((0, img.shape[1]))
ax.set_ylim((img.shape[0], 0))
plt.gca().set_aspect('equal')

In [ ]:
# subset trancsripts

def subset_transcripts(df, x_offset, y_offset, max_width, max_height):
    df = df[
        ((df.x ) >= (x_offset - 20)) &
        ((df.x ) <= (x_offset + max_width + 20)) &
        ((df.y ) >= (y_offset - 20 )) &
        ((df.y ) <= (y_offset + max_height + 20))
    ]
    return (df)

df = subset_transcripts(df, x_offset, y_offset, max_width, max_height)

## Create cell boundaries

Using alphashapes.


In [ ]:
from matplotlib.patches import Patch
def plot_segmentation(df, img, type: str = "cell_id"):

    print("... Plotting image")
    fig = plt.figure(figsize = (15,15))
    ax = fig.add_subplot(111)
    ax.imshow(
        img,
        vmin=np.percentile(img, 99)*0.1,
        vmax=np.percentile(img, 99)*1.1,
        cmap=sns.dark_palette("#bfcee3", reverse=False, as_cmap=True)
    )

    ax.set_xlim(tx, (tx+th))
    ax.set_ylim((ty+th), ty)

    add_boundaries(df, type, ax)
    ax.set_aspect('equal')
    return (fig, ax)

def add_boundaries(df, type,  ax, color=sns.color_palette()[3], fill=True, alpha=0.7):
    from matplotlib.patches import Patch
    def make_alphashape(points: pd.DataFrame, alpha: float):
        points = np.array(points)
        shape = alphashape(points, alpha=alpha)
        return shape

    print("... Calculating alphashapes")
    shapes = df[~(pd.isnull(df[type]))].groupby(type)[['x', 'y']].apply(make_alphashape, alpha=0.05)
    shapes = gpd.GeoSeries(shapes)

    if fill:
        shapes.apply(lambda x: transform(rescale_fun_2d, x)).plot(facecolor=color, edgecolor='none', alpha=0.2, ax=ax)
    shapes.apply(lambda x: transform(rescale_fun_2d, x)).plot(facecolor="none", edgecolor=color, alpha=alpha,  ax=ax)

    return ax



## Plot image

### Using the cellpose segmentation

In [ ]:
fig, ax = plot_segmentation(df, img, "cell_id")
legend_elements = [Patch(facecolor=sns.color_palette()[3] + (0.2,), 
                         edgecolor=sns.color_palette()[3] + (0.7,),
                         label='Cellpose')]
ax.legend(handles=legend_elements)

### Using Baysor segmentation

In [ ]:
fig, ax = plot_segmentation(df, img, "cell")
legend_elements = [Patch(facecolor=sns.color_palette()[3] + (0.2,), 
                         edgecolor=sns.color_palette()[3] + (0.7,),
                         label='Baysor')]
ax.legend(handles=legend_elements)

### Baysor and Cellpose

In [ ]:
fig, ax = plot_segmentation(df, img, "cell")
ax = add_boundaries(df, "cell_id", ax, color=sns.color_palette()[0], fill=False, alpha=0.7)
legend_elements = [Patch(facecolor=sns.color_palette()[3] + (0.2,), 
                         edgecolor=sns.color_palette()[3] + (0.7,),
                         label='Baysor'),
                   Patch(facecolor=sns.color_palette()[0] + (0.2,), 
                         edgecolor=sns.color_palette()[0] + (0.7,),
                         label='Cellpose')]
ax.legend(handles=legend_elements)